# App Reviews Analysis of Philippine Banks

---

This project seeks to analyze the performace of the iOS applications of the top 3 privately owned banks in the Philippines which are Banco De Oro (BDO), Bank of the Philippine Islands (BPI), and Metropolitan Bank & Trust Company (Metrobank) using Natural Language Processing or NLP.

The data is obtained from Apple App Store reviews. This project shows how you can derive insights generated by your users and make decisions based on these insights and data.

## Goal 3: Translate Non-English Reviews

---

From the language that we have detected in the previous notebook, we will now translate it so that the data are all in English.

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('datasets/df_wlanguages.csv', index_col='Unnamed: 0')
df.head()

,date,review,rating,isEdited,userName,title,developerResponse,bank_app,review_language,review_language_prob
0,8/21/2023 22:42,I tried this one because of the notification i...,1,False,Dence07,Useless app,NaN,Banco De Oro,eng_Latn,0.999971
1,6/29/2023 6:55,I am quite disappointed with the BDO App’s lac...,1,False,Domansss,SMH,NaN,Banco De Oro,eng_Latn,0.999866
2,9/18/2023 14:58,I have this app for a almost a month already a...,1,False,Je123DJ,Not User Friendly,NaN,Banco De Oro,eng_Latn,0.999952
3,8/18/2023 8:18,What are these apps you keep developing? Do th...,1,False,Rieyuxel,Higher a UX Designer!,NaN,Banco De Oro,eng_Latn,0.999986
4,2/3/2022 8:57,Since i have read the reviews and its not appe...,1,False,Abner61,TO MANY BDO APPS but useless.,NaN,Banco De Oro,eng_Latn,0.999955


In [3]:
df['review_language'].value_counts()

eng_Latn    3504
tgl_Latn     187
yue_Hant      61
kor_Hang      21
ceb_Latn       1
war_Latn       1
Name: review_language, dtype: int64

From the data above, there are few data from `yue_Hant`, `kor_Hang`, `ceb_Latn` and `war_Latn`. Translating these in CPU won't be much of a problem the differences are inisignificant compared when using GPU.

### Translation using CPU

---

Here is a simple flow to translate those underrepresented data into English. Throughout the notebook, we will translate the languages using `NLLB-200 600M variant`. It is a model created by Facebook to translate a wide range of the world's languages with high accuracy, especially those low-resource languages that are underrepresented in current technology.

In [4]:
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline

target_language = 'eng_Latn'

In [5]:
# Translate a text data in a pandas dataframe
def translate_dataframe(df, text_column, source_lang, target_lang):
    # Initialize the model and tokenizer
    checkpoint = 'facebook/nllb-200-distilled-600M'
    model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)
    tokenizer = AutoTokenizer.from_pretrained(checkpoint)
    
    # Create the translation pipeline
    translator = pipeline('translation', model=model, tokenizer=tokenizer, src_lang=source_lang, tgt_lang=target_lang, max_length=400)
    
    # Translate the text in the specified column and store in a new column
    df['translated_review'] = df[text_column].apply(lambda x: translator(x)[0]['translation_text'] if pd.notnull(x) else x)
    
    return df

In [6]:
# Create a separate dataframe for cebuano and waray languages
ceb_df = df[df['review_language'] =='ceb_Latn']
war_df = df[df['review_language'] =='war_Latn']

In [7]:
# Translate the languages by calling the function
transltd_df_ceb = translate_dataframe(ceb_df, 'review', 'ceb_Latn', 'eng_Latn')
transltd_df_war = translate_dataframe(war_df, 'review', 'war_Latn', 'eng_Latn')

/tmp/ipykernel_278/2413969712.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['translated_review'] = df[text_column].apply(lambda x: translator(x)[0]['translation_text'] if pd.notnull(x) else x)
/tmp/ipykernel_278/2413969712.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['translated_review'] = df[text_column].apply(lambda x: translator(x)[0]['translation_text'] if pd.notnull(x) else x)


In [8]:
transltd_df_ceb

,date,review,rating,isEdited,userName,title,developerResponse,bank_app,review_language,review_language_prob,translated_review
3562,2023-08-16 06:35:20,Hanggaw mani inyung mobile app maka sapot maka...,1,False,Nicki Minaj Khristine Hermosa,WORST MOBILE BANKING,"{'id': 38374117, 'body': ""Hi Nicki Minaj Khris...",Metropolitan Bank & Trust Company,ceb_Latn,0.975935,Until you run this mobile app you can wear unb...


In [9]:
transltd_df_war

,date,review,rating,isEdited,userName,title,developerResponse,bank_app,review_language,review_language_prob,translated_review
3647,2023-08-07 06:13:34,Buloka ani nga app oy jusko,1,False,tOikkz,Bulok na app,"{'id': 38180054, 'body': ""Hi tOikkz, we're sor...",Metropolitan Bank & Trust Company,war_Latn,0.71594,Use this app or juice


### Translation using GPU

---

Now here is where NLP tasks usually happens, it runs in parallel computations so that it is more efficient, and that is the task where GPUs reign supreme. Up until now, we are just using the CPU for the tasks that we have done so far, and that is not the correct way especially when dealing with large amounts of data with high computation throughput, but becuase our data is just small, we can get away with just by using CPU. 

Not on this prticualr task though, using this in CPU would take you a significant time (I gave up already when my kernel is not yet finished after 15 mins of run time). 

Becuase we are using models hosted in Huggingface, we need a better data type to run NLP tasks in a much more efficient manner, it is called dataset, which you can look for more information [here](https://huggingface.co/docs/datasets/loading).

The workflow for this NLP task using GPU would be like this:

In [10]:
# Create a separate dataframe for the major non-English languages
df_tgl = df[df['review_language'] == 'tgl_Latn']
df_yue = df[df['review_language'] == 'yue_Hant']
df_kor = df[df['review_language'] == 'kor_Hang']

Let's check if CUDA is avilable in this cloud GPU

In [11]:
import torch

# Check if CUDA is available
cuda_available = torch.cuda.is_available()

# Print whether CUDA is available
print(f"CUDA available: {cuda_available}")

# If CUDA is available, you can also print out the number of CUDA devices and their names
if cuda_available:
    print(f"Number of CUDA devices: {torch.cuda.device_count()}")
    for i in range(torch.cuda.device_count()):
        print(f"CUDA device {i}: {torch.cuda.get_device_name(i)}")

CUDA available: True
Number of CUDA devices: 1
CUDA device 0: Quadro P5000


Great! CUDA is available. Let's convert those language separated dataframes into a Huggingface dataset.

In [12]:
from datasets import Dataset

dataset_tgl = Dataset.from_pandas(df_tgl)
dataset_yue = Dataset.from_pandas(df_yue)
dataset_kor = Dataset.from_pandas(df_kor)

We will now recreate a new set of functions for this GPU task. The first function sets up the translation pipeline, and the second function conducts a batch translation.

In [13]:
# Define a function to set up the translation model and pipeline
def setup_translation_model(checkpoint, source_lang, target_lang):
    device = "cuda" if torch.cuda.is_available() else "cpu"
    model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint).to(device)
    tokenizer = AutoTokenizer.from_pretrained(checkpoint)
    translator = pipeline('translation', model=model, tokenizer=tokenizer, src_lang=source_lang, tgt_lang=target_lang, max_length=1000, device=0 if torch.cuda.is_available() else -1)
    return translator

In [14]:
# Initialize the translation pipeline
checkpoint = 'facebook/nllb-200-distilled-600M'
tgl_translator = setup_translation_model(checkpoint, 'tgl_Latn', target_language)
yue_translator = setup_translation_model(checkpoint, 'yue_Hant', target_language)
kor_translator = setup_translation_model(checkpoint, 'kor_Hang', target_language)

In [15]:
# Function to apply translation to each review in the dataset
def translate_batch(batch, translator):
    # The translator pipeline can handle a list of texts
    texts = batch['review']
    translations = translator(texts, max_length=1000)
    batch['translated_review'] = [translation['translation_text'] for translation in translations]
    return batch

In [16]:
%%time

# Translate all of the non-english language using the GPU
tgl_dataset_translated = dataset_tgl.map(lambda batch: translate_batch(batch, translator=tgl_translator), batched=True)
yue_dataset_translated = dataset_yue.map(lambda batch: translate_batch(batch, translator=yue_translator), batched=True)
kor_dataset_translated = dataset_kor.map(lambda batch: translate_batch(batch, translator=kor_translator), batched=True)

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

CPU times: user 3min 7s, sys: 36 s, total: 3min 43s
Wall time: 3min 43s


Now that was fast. If that was in CPU, you can even watch a TV series while waiting for it to finish.

Now let's convert the dataset back into a dataframe.

In [17]:
tgl_df = tgl_dataset_translated.to_pandas()
yue_df = yue_dataset_translated.to_pandas()
kor_df = kor_dataset_translated.to_pandas()

In converting it back to dataframe, additional column named `__index_level_0__` was created. This was our original index earlier, let's set it to be index in the dataset we had just converted.

In [18]:
# Set the index manually from the new column that was generated in the HF dataset
tgl_df.set_index('__index_level_0__', inplace=True)
yue_df.set_index('__index_level_0__', inplace=True)
kor_df.set_index('__index_level_0__', inplace=True)

# Remove the name of the index
tgl_df.index.name = ''
yue_df.index.name = ''
kor_df.index.name = ''

### Putting it all together

---

We are done now in translating all of the non-English reviews we have encountered, but it is so dispersed becuase we have created a new dataframe for each of the distinct languages that lid218e model had labelled.

Let's piece all of it together; we can use the index numbers so that we can insert it in the orignal data that we had. But we will not insert it in the original data itself, let's just create a copy so that the orignal data won't get modified.

In [19]:
copy_df = df.copy()
copy_df.head()

,date,review,rating,isEdited,userName,title,developerResponse,bank_app,review_language,review_language_prob
0,8/21/2023 22:42,I tried this one because of the notification i...,1,False,Dence07,Useless app,NaN,Banco De Oro,eng_Latn,0.999971
1,6/29/2023 6:55,I am quite disappointed with the BDO App’s lac...,1,False,Domansss,SMH,NaN,Banco De Oro,eng_Latn,0.999866
2,9/18/2023 14:58,I have this app for a almost a month already a...,1,False,Je123DJ,Not User Friendly,NaN,Banco De Oro,eng_Latn,0.999952
3,8/18/2023 8:18,What are these apps you keep developing? Do th...,1,False,Rieyuxel,Higher a UX Designer!,NaN,Banco De Oro,eng_Latn,0.999986
4,2/3/2022 8:57,Since i have read the reviews and its not appe...,1,False,Abner61,TO MANY BDO APPS but useless.,NaN,Banco De Oro,eng_Latn,0.999955


For all of the reviews that we had translated, it created a new column to store that information, that column is absent in the original data that we had. 

Let's create two new columns, one is an identifier that it didn't got translated and another one that stores non translated reviews.

In [20]:
# Identifier that the reviews didn't got translated
copy_df['is_translated'] = 'no'

# Stores the review as a copy paste to the translated_review 
copy_df['translated_review'] = copy_df['review']

In [21]:
copy_df.head(3)

,date,review,rating,isEdited,userName,title,developerResponse,bank_app,review_language,review_language_prob,is_translated,translated_review
0,8/21/2023 22:42,I tried this one because of the notification i...,1,False,Dence07,Useless app,NaN,Banco De Oro,eng_Latn,0.999971,no,I tried this one because of the notification i...
1,6/29/2023 6:55,I am quite disappointed with the BDO App’s lac...,1,False,Domansss,SMH,NaN,Banco De Oro,eng_Latn,0.999866,no,I am quite disappointed with the BDO App’s lac...
2,9/18/2023 14:58,I have this app for a almost a month already a...,1,False,Je123DJ,Not User Friendly,NaN,Banco De Oro,eng_Latn,0.999952,no,I have this app for a almost a month already a...


For all of the non-English language, let's combine it to be in a one dataframe. After that we will create an additional column which would serve as an indentifier that we had translated the review.

In [22]:
non_eng_df = pd.concat([transltd_df_ceb, transltd_df_war, tgl_df, yue_df, kor_df])
non_eng_df['is_translated'] = 'yes'

Let's fix the position of the translated review so that it matches the arrangement from the `copy_df` dataframe.

In [23]:
# Create a list of all column names except the one you want to move
non_moving_cols = [c for c in non_eng_df.columns if c != 'translated_review']

# Now add the column name that you want to move at the end of this list
non_moving_cols.append('translated_review')

# Reindex the DataFrame with the new column order
non_eng_df_fixed = non_eng_df[non_moving_cols]

In [24]:
non_eng_df_fixed.head()

,date,review,rating,isEdited,userName,title,developerResponse,bank_app,review_language,review_language_prob,is_translated,translated_review
3562,2023-08-16 06:35:20,Hanggaw mani inyung mobile app maka sapot maka...,1,False,Nicki Minaj Khristine Hermosa,WORST MOBILE BANKING,"{'id': 38374117, 'body': ""Hi Nicki Minaj Khris...",Metropolitan Bank & Trust Company,ceb_Latn,0.975935,yes,Until you run this mobile app you can wear unb...
3647,2023-08-07 06:13:34,Buloka ani nga app oy jusko,1,False,tOikkz,Bulok na app,"{'id': 38180054, 'body': ""Hi tOikkz, we're sor...",Metropolitan Bank & Trust Company,war_Latn,0.715940,yes,Use this app or juice
660,12/2/2021 18:06,"Ginawa naman yung steps para sa update, pero y...",1,False,chmpagnepapi,BDO!!!,None,Banco De Oro,tgl_Latn,0.995195,yes,The website does not accept special characters...
785,3/22/2023 4:11,Maganda yung app user friendly. Pero bakit iba...,5,False,Winang18,Nice for 10 pesos fee,None,Banco De Oro,tgl_Latn,0.997474,yes,"It's a very user-friendly app, but why is it a..."
887,8/25/2023 4:33,You created a new app only to show an error me...,1,False,Felician11,HASSLE,None,Banco De Oro,tgl_Latn,0.933858,yes,You created a new app only to show an error me...


Now the `copy_df` and `non_eng_df_fixed` have the same column content and arrangement, we can now replace the translated contents in the `non_eng_df_fixed` to be inserted in the `copy_df`.

In [25]:
# This will update the contents of copy_df with the corresponding rows from non_eng_df_fixed based on the index.
# This is changed inplace
copy_df.update(non_eng_df_fixed)

/tmp/ipykernel_278/3251718904.py:3: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  copy_df.update(non_eng_df_fixed)


### Double Check

---

Let's see if the translated rows got inserted in the `copy_df` which is initially a dataset fill of non-translated reviews

In [26]:
# Determie the number of rows for the translated reviews
non_eng_df_fixed.shape

(271, 12)

In [27]:
# See if the number matched in the updated copy_df
copy_df['is_translated'].value_counts()

no     3504
yes     271
Name: is_translated, dtype: int64

### Saving the new data

---

The insertion of the new translated reviews are successful, let's save the new dataframe containing those translations.

In [28]:
copy_df.to_csv('datasets/translated_df.csv')